In [11]:
import os
import pandas as pd
import dateparser
from pydantic import BaseModel
from typing import Literal
from openai import OpenAI
from dotenv import load_dotenv
import re


load_dotenv()
client = OpenAI()


class ColumnTypes(BaseModel):
    types: list[str]

def reader_agent(df: pd.DataFrame) -> list[str]:
    # 1. We changed the input parameter to 'df: pd.DataFrame'
    print("[Reader Agent] Reading cropped dataframe to classify columns...")
    
    # 2. REMOVED the pd.read_excel() line entirely because the data is already loaded!
    
    # 3. Grab the sample directly from the passed dataframe
    sample_data = df.head(5).to_dict(orient="list")
    
    prompt = f"""
    Analyze the following data sample from an Excel file.
    For each column, determine its data type based on the values.
    You must return a list where each element corresponds to a column from left to right.
    
    You are ONLY allowed to use these exact categories: "time", "money", "int", "string", "float", "name", "unknown".
    
    CRITICAL DEFINITIONS:
    - "time": Includes standard formats (2023-01-01, 14:30), timestamps, AND natural language dates (e.g., "first of january 2016", "Q1 2024", "yesterday"). If the core meaning represents a date or time, it is "time", NEVER "string".
    - "money": Includes currency symbols ($100, €50), accounting formats, or financial abbreviations (100 USD) and natural language money expressions ("100 dollars", "fifty euros"). If the core meaning represents a monetary value, it is "money", NEVER "string".
    - "int": Whole numbers without decimals.
    - "float": Numbers containing decimals.
    - "name": Proper nouns. This includes human names (John Smith, Smith, John), cities, states (Alabama), or company names.
    - "string": General text, sentences, descriptions, or specific codes (e.g., ID-4552) that have no mathematical or temporal value.
    - "unknown": Use this ONLY if the column is complete gibberish or you cannot confidently assign it to any other category.

    Data sample (Columns and their first 5 values):
    {sample_data}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a data classification agent."},
            {"role": "user", "content": prompt}
        ],
        response_format=ColumnTypes
    )
    
    types = response.choices[0].message.parsed.types
    print(f"     [Reader Agent Classification] {types}")
    return types

class TimeFormatDecision(BaseModel):
    reasoning: str
    target_format: Literal[
        "%H:%M", "%H:%M:%S", "%S", 
        "%d/%m/%Y", "%d/%m/%Y %H:%M", "%d/%m/%Y %H:%M:%S", 
        "%m/%Y", "%Y"
    ]

def execute_time_formatting(df: pd.DataFrame, col_name: str, target_format: str) -> pd.DataFrame:
    """The Tool used to physically alter the dataframe."""
    print(f"       [Tool Executing] Formatting '{col_name}' to '{target_format}'...")
    
    def parse_natural_language(date_str):
        if pd.isna(date_str):
            return pd.NaT
            
        clean_str = str(date_str).lower()
        replacements = {
            "first": "1st", "second": "2nd", "third": "3rd", 
            "fourth": "4th", "fifth": "5th", "sixth": "6th", 
            "seventh": "7th", "eighth": "8th", "ninth": "9th", 
            "tenth": "10th", "eleventh": "11th", "twelfth": "12th", 
            "thirteenth": "13th", "fourteenth": "14th", "fifteenth": "15th", 
            "sixteenth": "16th", "seventeenth": "17th", "eighteenth": "18th", 
            "nineteenth": "19th", "twentieth": "20th",
            "twenty-first": "21st", "twenty first": "21st",
            "twenty-second": "22nd", "twenty second": "22nd",
            "twenty-third": "23rd", "twenty third": "23rd",
            "twenty-fourth": "24th", "twenty fourth": "24th",
            "twenty-fifth": "25th", "twenty fifth": "25th",
            "twenty-sixth": "26th", "twenty sixth": "26th",
            "twenty-seventh": "27th", "twenty seventh": "27th",
            "twenty-eighth": "28th", "twenty eighth": "28th",
            "twenty-ninth": "29th", "twenty ninth": "29th",
            "thirtieth": "30th", 
            "thirty-first": "31st", "thirty first": "31st",
            "last": "last"
        }
        for word, num in replacements.items():
            clean_str = clean_str.replace(word, num)
            
        parsed = dateparser.parse(clean_str)
        return parsed if parsed else pd.NaT

    try:
        df[col_name] = df[col_name].apply(parse_natural_language)
        df[col_name] = df[col_name].dt.strftime(target_format)
        print(f"       [Tool Success] Column updated.")
    except Exception as e:
        print(f"       [Tool Error] Failed: {e}")
    return df

def time_agent_workflow(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"  -> [Time Agent] Taking control of column: '{col_name}'")
    sample_data = df[col_name].dropna().head(5).tolist()
    
    prompt = f"""
    Look at this sample of time/date data from the column '{col_name}'.
    Data sample: {sample_data}
    
    Determine the appropriate standardized format for this data based on its granularity.
    - Hours and minutes: "%H:%M"
    - Hours, minutes, and seconds: "%H:%M:%S"
    - Just seconds: "%S"
    - Specific dates: "%d/%m/%Y"
    - Date and time: "%d/%m/%Y %H:%M"
    - Date and exact time: "%d/%m/%Y %H:%M:%S"
    - Month and year: "%m/%Y"
    - Year only: "%Y"
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are an expert data formatting agent."},
            {"role": "user", "content": prompt}
        ],
        response_format=TimeFormatDecision
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [Time Agent Decision] {decision.reasoning}")
    
    df = execute_time_formatting(df, col_name, decision.target_format)
    return df


def orchestrator_router(df: pd.DataFrame, type_vector: list[str], original_filename: str):
    print(f"\n[Orchestrator] Data loaded. Delegating tasks...")
    # REMOVED: df = pd.read_excel(file_path) because df is already loaded and cropped!
    
    for col_name, col_type in zip(df.columns, type_vector):
        if col_type == "time":
            df = time_agent_workflow(df, col_name)
        elif col_type == "money":
            df = money_agent_workflow(df, col_name) 
        elif col_type == "int":
            df = int_agent_workflow(df, col_name)
        elif col_type == "float":
            # Added float logic here just in case you need it!
            df = float_agent_workflow(df, col_name)
        elif col_type == "name":
            df = name_agent_workflow(df, col_name)
        elif col_type in ["string", "unknown"]:
            print(f"  -> [Orchestrator] Bypassing '{col_name}' (Type: {col_type} requires no formatting)")

    desc_df = dataset_description_agent(df)
    
    # Save the file using pd.ExcelWriter to support multiple tabs
    output_path = "cleaned_" + original_filename
    
    # We use engine='openpyxl' to ensure it writes modern .xlsx files properly
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Save the actual data to the first sheet
        df.to_excel(writer, sheet_name="Cleaned_Data", index=False)
        
        # Save the LLM's descriptions to the second sheet
        desc_df.to_excel(writer, sheet_name="dataset_description", index=False)
        
    print(f"\n[Orchestrator] All tasks complete. Saved multi-sheet file to: {output_path}")

class MoneyFormatDecision(BaseModel):
    reasoning: str
    is_mixed_currency: bool  
    detected_currency: str   
    scale_decision: Literal["None", "Thousands", "Millions", "Billions"]
    decimal_separator: Literal[".", ","]


def execute_money_formatting(df: pd.DataFrame, col_name: str, decision: MoneyFormatDecision) -> pd.DataFrame:
    print(f"       [Tool Executing] Scale: {decision.scale_decision}, Mixed Currency: {decision.is_mixed_currency}...")
    
    def parse_money_string(val):
        if pd.isna(val):
            return pd.NA, ""
            
        val_str = str(val).lower().strip()
        original_str = str(val).strip() 
        
        # 1. Extract the currency symbol, code, or full word
        symbol_match = re.search(r'([\$€£¥]|(?:usd|eur|gbp|jpy|dollars?|euros?|pounds?|yen))', original_str, re.IGNORECASE)
        raw_symbol = symbol_match.group(1).lower() if symbol_match else ""
        
        # 2. Normalize the currency
        currency_map = {
            "dollar": "USD", "dollars": "USD", "$": "USD", "usd": "USD",
            "euro": "EUR", "euros": "EUR", "eur": "EUR", "€": "EUR",
            "pound": "GBP", "pounds": "GBP", "gbp": "GBP", "£": "GBP",
            "yen": "JPY", "jpy": "JPY", "¥": "JPY"
        }
        symbol = currency_map.get(raw_symbol, raw_symbol.upper())
        
        # 3. Handle International Decimals
        if decision.decimal_separator == ",":
            val_str = val_str.replace('.', '').replace(',', '.')
        else:
            val_str = val_str.replace(',', '')
            
        # DEFENSIVE SHIELD: Remove extra dots
        if val_str.count('.') > 1:
            parts = val_str.rsplit('.', 1)
            val_str = parts[0].replace('.', '') + '.' + parts[1]
            
        # 4. Extract the core number
        match = re.search(r'[\d\.]+', val_str)
        if not match:
            return pd.NA, symbol
        try:
            num = float(match.group())
        except ValueError:
            return pd.NA, symbol
            
        # 5. Apply word multipliers
        isolated_words = re.sub(r'[\d\.\,€\$£¥]', ' ', val_str).split()
        
        if any(w in isolated_words for w in ['billion', 'billions', 'bill', 'bil', 'b']):
            num *= 1_000_000_000
        elif any(w in isolated_words for w in ['million', 'millions', 'mill', 'mil', 'm']):
            num *= 1_000_000
        elif any(w in isolated_words for w in ['thousand', 'thousands', 'k']):
            num *= 1_000
        elif any(w in isolated_words for w in ['cent', 'cents']):
            num /= 100
            
        return num, symbol

    try:
        parsed_data = df[col_name].apply(parse_money_string)
        
        nums = [x[0] if isinstance(x, tuple) else pd.NA for x in parsed_data]
        symbols = [x[1] if isinstance(x, tuple) else "" for x in parsed_data]
        
        # Overwrite the original column with just the pure math numbers
        df[col_name] = nums
        
        # 6. Apply the Scale Decision to the numbers
        scale_suffix = ""
        if decision.scale_decision == "Billions":
            df[col_name] = df[col_name] / 1_000_000_000
            scale_suffix = "in billions"
        elif decision.scale_decision == "Millions":
            df[col_name] = df[col_name] / 1_000_000
            scale_suffix = "in millions"
        elif decision.scale_decision == "Thousands":
            df[col_name] = df[col_name] / 1_000
            scale_suffix = "in thousands"

        # 7. Final Formatting: Split column vs Single Currency Header
        if decision.is_mixed_currency:
            # Find exactly where the current column is located
            col_idx = df.columns.get_loc(col_name)
            
            # Create the new column name
            new_currency_col = f"{col_name}_currency"
            
            # Insert the symbols list as a new column directly to the right
            df.insert(loc=col_idx + 1, column=new_currency_col, value=symbols)
            
            # Rename original number column if scaling was applied
            if scale_suffix:
                new_col_name = f"{col_name} ({scale_suffix})"
                df.rename(columns={col_name: new_col_name}, inplace=True)
                print(f"       [Tool Success] Mixed currencies split into '{new_currency_col}'. Numbers renamed to '{new_col_name}'.")
            else:
                print(f"       [Tool Success] Mixed currencies split into '{new_currency_col}'.")
                
        else:
            # Single currency: Keep as floats, put currency in the header, no new column needed
            parts = []
            if decision.detected_currency and decision.detected_currency != "Unknown":
                parts.append(decision.detected_currency)
            if scale_suffix:
                parts.append(scale_suffix)
                
            if parts:
                header_addition = " ".join(parts)
                new_col_name = f"{col_name} ({header_addition})"
                df.rename(columns={col_name: new_col_name}, inplace=True)
                print(f"       [Tool Success] Floats extracted. Renamed to '{new_col_name}'.")

    except Exception as e:
        print(f"       [Tool Error] Failed: {e}")
        
    return df

def money_agent_workflow(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"  -> [Money Agent] Taking control of column: '{col_name}'")
    sample_data = df[col_name].dropna().head(10).tolist()
    
    prompt = f"""
    Look at this sample of financial data from the column '{col_name}'.
    Data sample: {sample_data}
    
    Your task:
    1. Identify the primary currency being used (e.g., $, USD, €, Yen, "dollars", "euros"). 
       - CRITICAL RULE: If a currency is specified even just once in the sample, and NO OTHER currencies are mentioned, assume that single currency applies to the entire column.
    2. Set `is_mixed_currency` to True ONLY if you see multiple DIFFERENT currencies (e.g., "dollars" in one row and "eur" in another).
    3. Determine the best scale ("None", "Thousands", "Millions", "Billions").
       - Evaluate the TRUE underlying numerical value. "100 million" means 100,000,000. 
       - If the true values are predominantly in the millions, you MUST choose "Millions".
    4. Identify the decimal separator used in the numbers ("." or ",").
       - WARNING: Commas that group thousands (like "200,000,000") are NOT decimal separators. If a comma groups thousands, the decimal separator is ".".
       - Only choose "," if the comma specifically separates fractional cents at the very end of the number (e.g., "1.500,00").
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a precise financial data standardization agent."},
            {"role": "user", "content": prompt}
        ],
        response_format=MoneyFormatDecision
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [Money Agent Decision] Mixed: {decision.is_mixed_currency} | Currency: {decision.detected_currency} | Scale: {decision.scale_decision}")
    
    df = execute_money_formatting(df, col_name, decision)
    return df


def execute_int_formatting(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"       [Tool Executing] Cleaning and truncating '{col_name}' to integers...")
    
    def parse_int(val):
        if pd.isna(val):
            return pd.NA
            
        val_str = str(val).lower().replace(',', '').strip()
        
        try:
            num = float(val_str)
            return int(num)
        except ValueError:
            return pd.NA

    try:
        df[col_name] = df[col_name].apply(parse_int)
        df[col_name] = df[col_name].astype('Int64')
        
        print(f"       [Tool Success] Column '{col_name}' safely truncated to integers.")
    except Exception as e:
        print(f"       [Tool Error] Failed to process integers: {e}")
        
    return df

def int_agent_workflow(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"  -> [Int Agent] Taking control of column: '{col_name}' (Bypassing LLM for deterministic math)")
    
    df = execute_int_formatting(df, col_name)
    return df

def execute_float_formatting(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"       [Tool Executing] Standardizing floats for '{col_name}'...")
    
    # 1. Clean the data and convert to pure floats
    def extract_float(val):
        if pd.isna(val):
            return pd.NA
        val_str = str(val).lower().replace(',', '').strip()
        try:
            return float(val_str)
        except ValueError:
            return pd.NA
            
    raw_floats = df[col_name].apply(extract_float)
    
    # 2. Determine the maximum number of decimal places in the column
    max_decimals = 0
    for val in raw_floats.dropna():
        # Convert float to string (e.g., 0.876 -> "0.876") and split at the dot
        parts = str(val).split('.')
        if len(parts) == 2:
            decimals = len(parts[1])
            if max_decimals < decimals:
                max_decimals = decimals
                
    # 3. Format every number to match the max_decimals length
    def pad_float(val):
        if pd.isna(val):
            return pd.NA
        # This dynamically creates a format rule like "{:.3f}"
        return f"{val:.{max_decimals}f}"
        
    df[col_name] = raw_floats.apply(pad_float)
    
    print(f"       [Tool Success] Floats standardized to {max_decimals} decimal places.")
    return df


def float_agent_workflow(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"  -> [Float Agent] Taking control of column: '{col_name}' (Bypassing LLM)")
    
    df = execute_float_formatting(df, col_name)
    return df



class NameFormatDecision(BaseModel):
    reasoning: str
    entity_type: Literal["Human Names", "Locations/Other"]
    dominant_format: Literal["First Last", "Last First", "N/A"]


def execute_name_formatting(df: pd.DataFrame, col_name: str, decision: NameFormatDecision) -> pd.DataFrame:
    print(f"       [Tool Executing] Cleaning names. Type: {decision.entity_type}, Format: {decision.dominant_format}...")
    
    def parse_name(val):
        if pd.isna(val):
            return pd.NA
            
        # 1. Standardize capitalization (e.g., "JOHN smith" -> "John Smith")
        clean_name = str(val).strip().title()
        
        # 2. If it's a Location/Other, we just return the title-cased string
        if decision.entity_type == "Locations/Other":
            return clean_name
            
        # 3. Handle Human Names
        # If there's a comma (e.g., "Smith, John"), split it and force "First Last"
        if "," in clean_name:
            parts = [p.strip() for p in clean_name.split(",")]
            if len(parts) == 2:
                return f"{parts[1]} {parts[0]}"
                
        # If the LLM determined the column is mostly "Last First" without commas (e.g., "Smith John")
        if decision.dominant_format == "Last First":
            parts = clean_name.split()
            if len(parts) == 2:
                # Flip it to "First Last"
                return f"{parts[1]} {parts[0]}"
                
        # Default fallback: return as-is (already title-cased)
        return clean_name

    try:
        df[col_name] = df[col_name].apply(parse_name)
        print(f"       [Tool Success] Column '{col_name}' standardized.")
    except Exception as e:
        print(f"       [Tool Error] Failed to process names: {e}")
        
    return df


def name_agent_workflow(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    print(f"  -> [Name Agent] Taking control of column: '{col_name}'")
    
    # Grab 10 rows to give the LLM enough pattern context
    sample_data = df[col_name].dropna().head(10).tolist()
    
    prompt = f"""
    Look at this sample of proper nouns from the column '{col_name}'.
    Data sample: {sample_data}
    
    Your task:
    1. Determine if this column primarily contains "Human Names" or "Locations/Other" (like cities, states, companies).
    2. If it is "Human Names", deduce the dominant structural format.
       - Are they mostly "First Last" (e.g., John Smith)?
       - Are they mostly "Last First" (e.g., Smith John)?
       - NOTE: If you see ambiguous names (like "Harper Taylor"), look at the other names in the sample to deduce the pattern.
    3. If it is "Locations/Other", select "N/A" for the format.
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a precise text standardization agent."},
            {"role": "user", "content": prompt}
        ],
        response_format=NameFormatDecision
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [Name Agent Decision] Type: {decision.entity_type} | Dominant Format: {decision.dominant_format}")
    
    df = execute_name_formatting(df, col_name, decision)
    return df

class HeaderDecision(BaseModel):
    reasoning: str
    header_row_index: int
    header_col_index: int

def header_detection_agent(file_path: str):
    print(f"\n[Header Agent] Scanning '{file_path}' for the true table coordinates...")
    
    try:
        if file_path.endswith('.csv'):
            df_raw = pd.read_csv(file_path, header=None, nrows=15)
        else:
            df_raw = pd.read_excel(file_path, header=None, nrows=15)
    except Exception as e:
        print(f"Error reading file for header detection: {e}")
        return 0, 0

    # Fill NaN values with empty strings so the LLM can easily see the "blank" cells
    df_raw = df_raw.fillna("")
    raw_sample = df_raw.to_dict(orient="records")
    
    prompt = f"""
    Look at the first 15 rows of this raw data file.
    Real-world files often have titles, export dates, or blank rows at the very top. They also frequently have blank columns on the left.
    
    Your task is to identify the 2D starting coordinate of the ACTUAL data table:
    1. `header_row_index`: The 0-based index of the row containing the column headers (e.g., 'Txn ID', 'Date', 'Amount').
    2. `header_col_index`: The 0-based index of the column where the actual data starts (ignoring empty/blank columns to the left).
    
    Raw data sample: {raw_sample}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a data parsing agent specialized in finding table structures."},
            {"role": "user", "content": prompt}
        ],
        response_format=HeaderDecision
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [Header Agent Decision] Table starts at Row {decision.header_row_index}, Column {decision.header_col_index}.")
    print(f"     [Reasoning] {decision.reasoning}")
    
    return decision.header_row_index, decision.header_col_index



class MissingDataDecision(BaseModel):
    reasoning: str
    custom_na_strings_to_wipe: list[str]  # e.g., ["-", ".", "?"]
    remove_completely_empty_rows: bool
    remove_completely_empty_columns: bool


def execute_missing_data_cleaning(df: pd.DataFrame, decision: MissingDataDecision) -> pd.DataFrame:
    print(f"       [Tool Executing] Cleaning custom NAs, empty rows, and empty columns...")
    try:
        # 1. Wipe custom NA strings
        if decision.custom_na_strings_to_wipe:
            def wipe_custom_na(val):
                if isinstance(val, str) and val.strip() in decision.custom_na_strings_to_wipe:
                    return pd.NA
                return val
            
            df = df.map(wipe_custom_na)
            print(f"       [Tool Success] Wiped strings: {decision.custom_na_strings_to_wipe}")
        
        # 2. Remove completely empty rows
        if decision.remove_completely_empty_rows:
            initial_rows = len(df)
            df = df.dropna(axis=0, how='all')
            rows_removed = initial_rows - len(df)
            if rows_removed > 0:
                print(f"       [Tool Success] Dropped {rows_removed} completely empty rows.")
                
        # 3. Remove completely empty columns
        if decision.remove_completely_empty_columns:
            initial_cols = len(df.columns)
            # axis=1 tells Pandas to drop columns instead of rows
            df = df.dropna(axis=1, how='all')
            cols_removed = initial_cols - len(df.columns)
            if cols_removed > 0:
                print(f"       [Tool Success] Dropped {cols_removed} completely empty columns.")
            
    except Exception as e:
        print(f"       [Tool Error] Failed to clean missing data: {e}")
        
    return df

import string

def na_agent_workflow(df: pd.DataFrame) -> pd.DataFrame:
    print(f"\n[NA Agent] Scanning dataframe for custom missing values, empty rows, and empty columns...")
    
    # Pre-scan the data for 1-2 character strings that are pure punctuation
    potential_nas = set()
    for col in df.columns:
        str_vals = df[col].dropna().astype(str)
        for val in str_vals:
            val = val.strip()
            if 0 < len(val) <= 2 and all(c in string.punctuation for c in val):
                potential_nas.add(val)
                
    sample_data = df.head(10).to_dict(orient="records")
    
    prompt = f"""
    Analyze this dataset sample to identify how missing data is represented.
    Pandas has already handled standard 'NaN' and 'N/A' automatically.
    
    However, we detected these specific punctuation-only strings in the dataset: {list(potential_nas)}
    
    Your task:
    1. Evaluate if any of these strings (like "-", ".") are being used as placeholders for missing data. If so, add them to `custom_na_strings_to_wipe`.
    2. Decide if completely empty rows (rows where every single column is missing) should be removed. For standard tables, this is usually True.
    3. Decide if completely empty columns (columns where every single row is missing) should be removed. For standard tables, this is usually True.
    
    Data Sample:
    {sample_data}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a data cleaning agent focused on missing values."},
            {"role": "user", "content": prompt}
        ],
        response_format=MissingDataDecision
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [NA Agent Decision] Wipe strings: {decision.custom_na_strings_to_wipe} | Drop rows: {decision.remove_completely_empty_rows} | Drop cols: {decision.remove_completely_empty_columns}")
    
    df = execute_missing_data_cleaning(df, decision)
    return df

class FeatureDescription(BaseModel):
    feature_name: str
    inferred_data_type: str  # e.g., "Continuous Numeric", "Categorical", "Datetime"
    description: str

class DatasetDescription(BaseModel):
    general_summary: str
    features: list[FeatureDescription]

def dataset_description_agent(df: pd.DataFrame) -> pd.DataFrame:
    print(f"\n  -> [Description Agent] Analyzing the final dataset to generate feature documentation...")
    
    # Grab a sample of the cleaned data
    sample_data = df.head(5).to_dict(orient="list")
    
    prompt = f"""
    Analyze the following sample of a cleaned dataset.
    Your task is to generate a comprehensive data dictionary.
    
    1. Provide a 1-2 sentence `general_summary` of what this dataset represents.
    2. For every single column in the dataset, create a `FeatureDescription` detailing:
       - The exact column name.
       - The conceptual data type (e.g., Categorical, Datetime, Continuous Numeric, Text).
       - A clear, concise description of what the data represents based on the column name and the values.
    
    Data Sample:
    {sample_data}
    """

    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are an expert data analyst and documentation agent."},
            {"role": "user", "content": prompt}
        ],
        response_format=DatasetDescription
    )
    
    decision = response.choices[0].message.parsed
    print(f"     [Description Agent] Summary: {decision.general_summary}")
    
    # Convert the LLM's Pydantic objects into a list of dictionaries for Pandas
    description_records = []
    for f in decision.features:
        description_records.append({
            "Feature Name": f.feature_name,
            "Conceptual Data Type": f.inferred_data_type,
            "Description": f.description
        })
        
    # Create the DataFrame for the new Excel sheet
    description_df = pd.DataFrame(description_records)
    print(f"     [Description Agent] Successfully generated dictionary for {len(description_records)} features.")
    
    return description_df



if __name__ == "__main__":
    test_file = "case_A1_sales_light_dirty_input.xlsx"
    
    print("--- STARTING AGENTIC PIPELINE ---")
    
    # 1. SCOUT
    header_row, header_col = header_detection_agent(test_file)
    
    # 2. LOAD
    print(f"\n[Orchestrator] Loading file starting at row {header_row}...")
    if test_file.endswith('.csv'):
        df = pd.read_csv(test_file, header=header_row)
    else:
        df = pd.read_excel(test_file, header=header_row)
        
    # 3. CROP
    if header_col > 0:
        print(f"[Orchestrator] Cropping {header_col} empty columns from the left...")
        df = df.iloc[:, header_col:]
        
    # 4. SWEEP: The NA Agent purges custom nulls and empty rows
    df = na_agent_workflow(df)
    
    # 5. READ: Pass the sanitized dataframe to the Reader Agent
    classified_types = reader_agent(df)
    
    # 6. ROUTE: Pass to Orchestrator
    orchestrator_router(df, classified_types, test_file)

--- STARTING AGENTIC PIPELINE ---

[Header Agent] Scanning 'case_A1_sales_light_dirty_input.xlsx' for the true table coordinates...
     [Header Agent Decision] Table starts at Row 3, Column 0.
     [Reasoning] The data sample reveals that the first three rows contain meta-information and are not part of the data table, as they include titles, notes, and blank rows. 

The row with index 3 contains the actual column headers for the data table, identified by standard labels like 'Txn ID', 'Date', etc. These headers clearly mark the start of structured data.

The actual data in the sample starts from column 0, with 'Txn ID' being the first column in the dataset. Thus, the 2D coordinate indicating the start of the data table is (3, 0).

[Orchestrator] Loading file starting at row 3...

[NA Agent] Scanning dataframe for custom missing values, empty rows, and empty columns...
     [Header Agent Decision] Table starts at Row 3, Column 0.
     [Reasoning] The data sample reveals that the first

In [2]:
["first of january 2016", "january second 2016", "yesterday"]

['first of january 2016', 'january second 2016', 'yesterday']

In [ ]:
import asyncio
import sys
import string
from pathlib import Path
from typing import Literal
from agents import Agent, Runner, function_tool
from agents.mcp import MCPServerStdio
import pandas as pd
import dateparser
import re

# 1. Define the MCP Server for Data Formatting Tools
# We will write this to a separate file and run it as a subprocess
mcp_server_code = r"""
import pandas as pd
import dateparser
import re
import string
from typing import Literal
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("data-formatting-tools")

def _read_file(file_path: str, header: int = 0) -> pd.DataFrame:
    '''Helper to read both CSV and Excel files.'''
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path, header=header)
    else:
        return pd.read_excel(file_path, header=header)

def _save_file(df: pd.DataFrame, file_path: str, index: bool = False):
    '''Helper to save both CSV and Excel files.'''
    if file_path.endswith('.csv'):
        df.to_csv(file_path, index=index)
    else:
        df.to_excel(file_path, index=index)

@mcp.tool()
def execute_header_detection(file_path: str) -> str:
    '''Detect the true header row and starting column of a data table in a file.
    Returns the header_row_index and header_col_index as a string like "row:2,col:1".
    The raw preview of the first 15 rows is also returned for context.
    
    Args:
        file_path: Path to the Excel or CSV file.
    '''
    try:
        if file_path.endswith('.csv'):
            df_raw = pd.read_csv(file_path, header=None, nrows=15)
        else:
            df_raw = pd.read_excel(file_path, header=None, nrows=15)
        df_raw = df_raw.fillna("")
        raw_sample = df_raw.to_dict(orient="records")
        return f"RAW_PREVIEW: {raw_sample}"
    except Exception as e:
        return f"Error reading file for header detection: {e}"

@mcp.tool()
def apply_header_and_crop(file_path: str, header_row_index: int, header_col_index: int) -> str:
    '''Re-read the file with the correct header row and crop empty columns from the left.
    Overwrites the file with the properly loaded and cropped data.
    
    Args:
        file_path: Path to the Excel or CSV file.
        header_row_index: The 0-based row index of the true header.
        header_col_index: The 0-based column index where data starts.
    '''
    try:
        df = _read_file(file_path, header=header_row_index)
        if header_col_index > 0:
            df = df.iloc[:, header_col_index:]
        _save_file(df, file_path)
        return f"Successfully applied header at row {header_row_index}, cropped {header_col_index} columns. Shape: {df.shape}. Columns: {list(df.columns)}"
    except Exception as e:
        return f"Error applying header/crop: {e}"

@mcp.tool()
def execute_na_cleaning(
    file_path: str,
    custom_na_strings_to_wipe: list[str],
    remove_completely_empty_rows: bool,
    remove_completely_empty_columns: bool
) -> str:
    '''Clean missing data: wipe custom NA placeholder strings, remove empty rows/columns.
    
    Args:
        file_path: Path to the Excel or CSV file.
        custom_na_strings_to_wipe: List of strings to treat as NA (e.g., ["-", ".", "?"]).
        remove_completely_empty_rows: Whether to drop rows where all values are missing.
        remove_completely_empty_columns: Whether to drop columns where all values are missing.
    '''
    try:
        df = _read_file(file_path)
        messages = []
        
        if custom_na_strings_to_wipe:
            def wipe_custom_na(val):
                if isinstance(val, str) and val.strip() in custom_na_strings_to_wipe:
                    return pd.NA
                return val
            df = df.map(wipe_custom_na)
            messages.append(f"Wiped custom NA strings: {custom_na_strings_to_wipe}")
        
        if remove_completely_empty_rows:
            initial_rows = len(df)
            df = df.dropna(axis=0, how='all')
            rows_removed = initial_rows - len(df)
            if rows_removed > 0:
                messages.append(f"Dropped {rows_removed} completely empty rows.")
                
        if remove_completely_empty_columns:
            initial_cols = len(df.columns)
            df = df.dropna(axis=1, how='all')
            cols_removed = initial_cols - len(df.columns)
            if cols_removed > 0:
                messages.append(f"Dropped {cols_removed} completely empty columns.")
        
        _save_file(df, file_path)
        return f"NA cleaning complete. {'; '.join(messages)}. Shape: {df.shape}"
    except Exception as e:
        return f"Error cleaning NAs: {e}"

@mcp.tool()
def detect_potential_na_strings(file_path: str) -> str:
    '''Pre-scan the dataset for short punctuation-only strings that might be NA placeholders.
    Also returns a sample of the first 10 rows for context.
    
    Args:
        file_path: Path to the Excel or CSV file.
    '''
    try:
        df = _read_file(file_path)
        potential_nas = set()
        for col in df.columns:
            str_vals = df[col].dropna().astype(str)
            for val in str_vals:
                val = val.strip()
                if 0 < len(val) <= 2 and all(c in string.punctuation for c in val):
                    potential_nas.add(val)
        
        sample_data = df.head(10).to_dict(orient="records")
        return f"POTENTIAL_NAS: {list(potential_nas)}\nSAMPLE: {sample_data}"
    except Exception as e:
        return f"Error detecting NAs: {e}"

@mcp.tool()
def execute_time_formatting(
    file_path: str, 
    col_name: str, 
    target_format: Literal["%H:%M", "%H:%M:%S", "%S", "%d/%m/%Y", "%d/%m/%Y %H:%M", "%d/%m/%Y %H:%M:%S", "%m/%Y", "%Y"]
) -> str:
    '''Format a time/date column in a file to a specific target format.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to format.
        target_format: The target strftime format.
    '''
    try:
        df = _read_file(file_path)
        
        def parse_natural_language(date_str):
            if pd.isna(date_str):
                return pd.NaT
                
            clean_str = str(date_str).lower()
            replacements = {
                "first": "1st", "second": "2nd", "third": "3rd", 
                "fourth": "4th", "fifth": "5th", "sixth": "6th", 
                "seventh": "7th", "eighth": "8th", "ninth": "9th", 
                "tenth": "10th", "eleventh": "11th", "twelfth": "12th", 
                "thirteenth": "13th", "fourteenth": "14th", "fifteenth": "15th", 
                "sixteenth": "16th", "seventeenth": "17th", "eighteenth": "18th", 
                "nineteenth": "19th", "twentieth": "20th",
                "twenty-first": "21st", "twenty first": "21st",
                "twenty-second": "22nd", "twenty second": "22nd",
                "twenty-third": "23rd", "twenty third": "23rd",
                "twenty-fourth": "24th", "twenty fourth": "24th",
                "twenty-fifth": "25th", "twenty fifth": "25th",
                "twenty-sixth": "26th", "twenty sixth": "26th",
                "twenty-seventh": "27th", "twenty seventh": "27th",
                "twenty-eighth": "28th", "twenty eighth": "28th",
                "twenty-ninth": "29th", "twenty ninth": "29th",
                "thirtieth": "30th", 
                "thirty-first": "31st", "thirty first": "31st",
                "last": "last"
            }
            for word, num in replacements.items():
                clean_str = clean_str.replace(word, num)
                
            parsed = dateparser.parse(clean_str)
            return parsed if parsed else pd.NaT

        df[col_name] = df[col_name].apply(parse_natural_language)
        df[col_name] = df[col_name].dt.strftime(target_format)
        _save_file(df, file_path)
        return f"Successfully formatted column '{col_name}' to '{target_format}'."
    except Exception as e:
        return f"Error formatting time: {e}"

@mcp.tool()
def execute_money_formatting(
    file_path: str, 
    col_name: str, 
    is_mixed_currency: bool, 
    detected_currency: str, 
    scale_decision: Literal["None", "Thousands", "Millions", "Billions"], 
    decimal_separator: Literal[".", ","]
) -> str:
    '''Format a money/financial column in a file. For mixed currencies, a separate 
    currency column is inserted to the right. For single currencies, the currency 
    code is added to the column header.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to format.
        is_mixed_currency: True if multiple currencies are present.
        detected_currency: The primary currency detected (e.g., 'USD', 'EUR').
        scale_decision: The scale to apply.
        decimal_separator: The decimal separator used in the raw data.
    '''
    try:
        df = _read_file(file_path)
        
        def parse_money_string(val):
            if pd.isna(val):
                return pd.NA, ""
                
            val_str = str(val).lower().strip()
            original_str = str(val).strip() 
            
            symbol_match = re.search(r'([\$€£¥]|(?:usd|eur|gbp|jpy|dollars?|euros?|pounds?|yen))', original_str, re.IGNORECASE)
            raw_symbol = symbol_match.group(1).lower() if symbol_match else ""
            
            currency_map = {
                "dollar": "USD", "dollars": "USD", "$": "USD", "usd": "USD",
                "euro": "EUR", "euros": "EUR", "eur": "EUR", "€": "EUR",
                "pound": "GBP", "pounds": "GBP", "gbp": "GBP", "£": "GBP",
                "yen": "JPY", "jpy": "JPY", "¥": "JPY"
            }
            symbol = currency_map.get(raw_symbol, raw_symbol.upper())
            
            if decimal_separator == ",":
                val_str = val_str.replace('.', '').replace(',', '.')
            else:
                val_str = val_str.replace(',', '')
                
            if val_str.count('.') > 1:
                parts = val_str.rsplit('.', 1)
                val_str = parts[0].replace('.', '') + '.' + parts[1]
                
            match = re.search(r'[\d\.]+', val_str)
            if not match:
                return pd.NA, symbol
            try:
                num = float(match.group())
            except ValueError:
                return pd.NA, symbol
                
            isolated_words = re.sub(r'[\d\.\,€\$£¥]', ' ', val_str).split()
            
            if any(w in isolated_words for w in ['billion', 'billions', 'bill', 'bil', 'b']):
                num *= 1_000_000_000
            elif any(w in isolated_words for w in ['million', 'millions', 'mill', 'mil', 'm']):
                num *= 1_000_000
            elif any(w in isolated_words for w in ['thousand', 'thousands', 'k']):
                num *= 1_000
            elif any(w in isolated_words for w in ['cent', 'cents']):
                num /= 100
                
            return num, symbol

        parsed_data = df[col_name].apply(parse_money_string)
        nums = [x[0] if isinstance(x, tuple) else pd.NA for x in parsed_data]
        symbols = [x[1] if isinstance(x, tuple) else "" for x in parsed_data]
        
        df[col_name] = nums
        
        scale_suffix = ""
        if scale_decision == "Billions":
            df[col_name] = df[col_name] / 1_000_000_000
            scale_suffix = "in billions"
        elif scale_decision == "Millions":
            df[col_name] = df[col_name] / 1_000_000
            scale_suffix = "in millions"
        elif scale_decision == "Thousands":
            df[col_name] = df[col_name] / 1_000
            scale_suffix = "in thousands"

        if is_mixed_currency:
            # Insert a separate currency column to the right (matching Cell 1 logic)
            col_idx = df.columns.get_loc(col_name)
            new_currency_col = f"{col_name}_currency"
            df.insert(loc=col_idx + 1, column=new_currency_col, value=symbols)
            
            if scale_suffix:
                new_col_name = f"{col_name} ({scale_suffix})"
                df.rename(columns={col_name: new_col_name}, inplace=True)
        else:
            parts = []
            if detected_currency and detected_currency != "Unknown":
                parts.append(detected_currency)
            if scale_suffix:
                parts.append(scale_suffix)
                
            if parts:
                header_addition = " ".join(parts)
                new_col_name = f"{col_name} ({header_addition})"
                df.rename(columns={col_name: new_col_name}, inplace=True)

        _save_file(df, file_path)
        return f"Successfully formatted money column '{col_name}'."
    except Exception as e:
        return f"Error formatting money: {e}"

@mcp.tool()
def execute_int_formatting(file_path: str, col_name: str) -> str:
    '''Clean and truncate a column to integers.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to format.
    '''
    try:
        df = _read_file(file_path)
        def parse_int(val):
            if pd.isna(val):
                return pd.NA
            val_str = str(val).lower().replace(',', '').strip()
            try:
                num = float(val_str)
                return int(num)
            except ValueError:
                return pd.NA

        df[col_name] = df[col_name].apply(parse_int)
        df[col_name] = df[col_name].astype('Int64')
        _save_file(df, file_path)
        return f"Successfully formatted integer column '{col_name}'."
    except Exception as e:
        return f"Error formatting integers: {e}"

@mcp.tool()
def execute_float_formatting(file_path: str, col_name: str) -> str:
    '''Standardize floats for a column.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to format.
    '''
    try:
        df = _read_file(file_path)
        def extract_float(val):
            if pd.isna(val):
                return pd.NA
            val_str = str(val).lower().replace(',', '').strip()
            try:
                return float(val_str)
            except ValueError:
                return pd.NA
                
        raw_floats = df[col_name].apply(extract_float)
        
        max_decimals = 0
        for val in raw_floats.dropna():
            parts = str(val).split('.')
            if len(parts) == 2:
                decimals = len(parts[1])
                if max_decimals < decimals:
                    max_decimals = decimals
                    
        def pad_float(val):
            if pd.isna(val):
                return pd.NA
            return f"{val:.{max_decimals}f}"
            
        df[col_name] = raw_floats.apply(pad_float)
        _save_file(df, file_path)
        return f"Successfully formatted float column '{col_name}' to {max_decimals} decimal places."
    except Exception as e:
        return f"Error formatting floats: {e}"

@mcp.tool()
def execute_name_formatting(
    file_path: str, 
    col_name: str, 
    entity_type: Literal["Human Names", "Locations/Other"], 
    dominant_format: Literal["First Last", "Last First", "N/A"]
) -> str:
    '''Standardize proper nouns/names in a column.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to format.
        entity_type: 'Human Names' or 'Locations/Other'.
        dominant_format: 'First Last', 'Last First', or 'N/A'.
    '''
    try:
        df = _read_file(file_path)
        def parse_name(val):
            if pd.isna(val):
                return pd.NA
            clean_name = str(val).strip().title()
            if entity_type == "Locations/Other":
                return clean_name
            if "," in clean_name:
                parts = [p.strip() for p in clean_name.split(",")]
                if len(parts) == 2:
                    return f"{parts[1]} {parts[0]}"
            if dominant_format == "Last First":
                parts = clean_name.split()
                if len(parts) == 2:
                    return f"{parts[1]} {parts[0]}"
            return clean_name

        df[col_name] = df[col_name].apply(parse_name)
        _save_file(df, file_path)
        return f"Successfully formatted name column '{col_name}'."
    except Exception as e:
        return f"Error formatting names: {e}"

@mcp.tool()
def execute_dataset_description(file_path: str, general_summary: str, features_json: str) -> str:
    '''Save a dataset description as a second sheet in the Excel file.
    The cleaned data goes to "Cleaned_Data" sheet and the description goes to "dataset_description" sheet.
    
    Args:
        file_path: Path to the Excel file.
        general_summary: A 1-2 sentence summary of the dataset.
        features_json: A JSON string representing a list of objects with keys "Feature Name", "Conceptual Data Type", "Description".
    '''
    import json
    try:
        df = _read_file(file_path)
        features = json.loads(features_json)
        desc_df = pd.DataFrame(features)
        
        if file_path.endswith('.csv'):
            # For CSV, save description as a separate file
            desc_path = file_path.replace('.csv', '_description.csv')
            df.to_csv(file_path, index=False)
            desc_df.to_csv(desc_path, index=False)
            return f"Saved cleaned data to '{file_path}' and description to '{desc_path}'."
        else:
            with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name="Cleaned_Data", index=False)
                desc_df.to_excel(writer, sheet_name="dataset_description", index=False)
            return f"Saved multi-sheet file to '{file_path}' with Cleaned_Data and dataset_description sheets."
    except Exception as e:
        return f"Error saving description: {e}"

if __name__ == "__main__":
    mcp.run(transport="stdio")
"""

with open("data_formatting_server.py", "w") as f:
    f.write(mcp_server_code)

# 2. Define local function tools for reading data
@function_tool
def read_column_sample(file_path: str, col_name: str, n: int = 10) -> str:
    """Read a sample of data from a specific column in an Excel or CSV file.
    
    Args:
        file_path: Path to the Excel or CSV file.
        col_name: Name of the column to sample.
        n: Number of rows to sample.
    """
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        if col_name not in df.columns:
            return f"Column '{col_name}' not found. Available columns: {list(df.columns)}"
        sample = df[col_name].dropna().head(n).tolist()
        return str(sample)
    except Exception as e:
        return f"Error reading sample: {e}"

@function_tool
def read_data_sample(file_path: str, n: int = 5) -> str:
    """Read a sample of the entire dataset from an Excel or CSV file.
    
    Args:
        file_path: Path to the Excel or CSV file.
        n: Number of rows to sample.
    """
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        sample = df.head(n).to_dict(orient="list")
        return str(sample)
    except Exception as e:
        return f"Error reading sample: {e}"

@function_tool
def get_columns(file_path: str) -> str:
    """Get the list of columns in an Excel or CSV file.
    
    Args:
        file_path: Path to the Excel or CSV file.
    """
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        return str(list(df.columns))
    except Exception as e:
        return f"Error reading columns: {e}"

# 3. Define the Agents

# --- HEADER DETECTION AGENT (NEW — matches Cell 1's header_detection_agent) ---
header_agent = Agent(
    name="Header Detection Agent",
    instructions=(
        "You are a data parsing agent specialized in finding table structures.\n"
        "Real-world files often have titles, export dates, or blank rows at the very top. "
        "They also frequently have blank columns on the left.\n\n"
        "STEP 1: Use the `execute_header_detection` MCP tool to get a raw preview of the first 15 rows.\n"
        "STEP 2: Analyze the raw preview to identify the 2D starting coordinate of the ACTUAL data table:\n"
        "   - `header_row_index`: The 0-based index of the row containing the column headers (e.g., 'Txn ID', 'Date', 'Amount').\n"
        "   - `header_col_index`: The 0-based index of the column where the actual data starts (ignoring empty/blank columns to the left).\n"
        "STEP 3: Use the `apply_header_and_crop` MCP tool to re-read the file with the correct header and crop empty columns.\n\n"
        "Return a summary of what you found and applied."
    ),
    model="gpt-4o-2024-08-06",
)

# --- NA AGENT (NEW — matches Cell 1's na_agent_workflow) ---
na_agent = Agent(
    name="NA Agent",
    instructions=(
        "You are a data cleaning agent focused on missing values.\n"
        "Pandas has already handled standard 'NaN' and 'N/A' automatically.\n\n"
        "STEP 1: Use the `detect_potential_na_strings` MCP tool to scan for punctuation-only strings "
        "that might be NA placeholders, and to get a sample of the data.\n"
        "STEP 2: Evaluate if any of these strings (like '-', '.') are being used as placeholders for missing data.\n"
        "STEP 3: Use the `execute_na_cleaning` MCP tool with your decisions:\n"
        "   - `custom_na_strings_to_wipe`: list of strings to treat as NA\n"
        "   - `remove_completely_empty_rows`: True for standard tables\n"
        "   - `remove_completely_empty_columns`: True for standard tables\n\n"
        "Return a summary of what was cleaned."
    ),
    model="gpt-4o-2024-08-06",
)

# --- READER AGENT ---
reader_agent = Agent(
    name="Reader Agent",
    instructions=(
        "You are a precise data analysis agent. Your job is to classify columns in a dataset.\n"
        "Use the `read_data_sample` tool to get a sample of the data (first 5 rows).\n"
        "For each column, determine its data type based on the values.\n"
        "You are ONLY allowed to use these exact categories: 'time', 'money', 'int', 'string', 'float', 'name', 'unknown'.\n\n"
        "CRITICAL DEFINITIONS:\n"
        "- 'time': Includes standard formats (2023-01-01, 14:30), timestamps, AND natural language dates (e.g., 'first of january 2016', 'Q1 2024', 'yesterday'). If the core meaning represents a date or time, it is 'time', NEVER 'string'.\n"
        "- 'money': Includes currency symbols ($100, €50), accounting formats, or financial abbreviations (100 USD) and natural language money expressions ('100 dollars', 'fifty euros'). If the core meaning represents a monetary value, it is 'money', NEVER 'string'.\n"
        "- 'int': Whole numbers without decimals.\n"
        "- 'float': Numbers containing decimals.\n"
        "- 'name': Proper nouns. This includes human names (John Smith, Smith, John), cities, states (Alabama), or company names.\n"
        "- 'string': General text, sentences, descriptions, or specific codes (e.g., ID-4552) that have no mathematical or temporal value.\n"
        "- 'unknown': Use this ONLY if the column is complete gibberish or you cannot confidently assign it to any other category.\n\n"
        "IMPORTANT: You MUST return your result as a JSON object mapping each column name to its classified type.\n"
        "Example format: {\"Column A\": \"time\", \"Column B\": \"money\", \"Column C\": \"int\"}\n"
        "The order must match the columns from left to right. Return ONLY this JSON mapping, nothing else."
    ),
    tools=[read_data_sample, get_columns],
    model="gpt-4o-2024-08-06",
)

# --- TIME AGENT ---
time_agent = Agent(
    name="Time Agent",
    instructions=(
        "You are an expert data formatting agent specializing in time and dates.\n"
        "When given a file and a column name, first use `read_column_sample` to look at the data.\n"
        "Determine the appropriate standardized format for this data based on its granularity:\n"
        "- Hours and minutes: '%H:%M'\n"
        "- Hours, minutes, and seconds: '%H:%M:%S'\n"
        "- Just seconds: '%S'\n"
        "- Specific dates: '%d/%m/%Y'\n"
        "- Date and time: '%d/%m/%Y %H:%M'\n"
        "- Date and exact time: '%d/%m/%Y %H:%M:%S'\n"
        "- Month and year: '%m/%Y'\n"
        "- Year only: '%Y'\n"
        "Then use the `execute_time_formatting` MCP tool to apply the format.\n"
        "You MUST pass the file_path, col_name, and target_format to the tool."
    ),
    model="gpt-4o-2024-08-06",
)

# --- MONEY AGENT ---
money_agent = Agent(
    name="Money Agent",
    instructions=(
        "You are a precise financial data standardization agent.\n"
        "When given a file and a column name, first use `read_column_sample` to look at the data (request at least 10 rows).\n"
        "Your task:\n"
        "1. Identify the primary currency being used (e.g., $, USD, €, Yen, 'dollars', 'euros').\n"
        "   - CRITICAL RULE: If a currency is specified even just once in the sample, and NO OTHER currencies are mentioned, assume that single currency applies to the entire column.\n"
        "2. Set `is_mixed_currency` to True ONLY if you see multiple DIFFERENT currencies (e.g., 'dollars' in one row and 'eur' in another).\n"
        "3. Determine the best scale ('None', 'Thousands', 'Millions', 'Billions').\n"
        "   - Evaluate the TRUE underlying numerical value. '100 million' means 100,000,000.\n"
        "   - If the true values are predominantly in the millions, you MUST choose 'Millions'.\n"
        "4. Identify the decimal separator used in the numbers ('.' or ',').\n"
        "   - WARNING: Commas that group thousands (like '200,000,000') are NOT decimal separators. If a comma groups thousands, the decimal separator is '.'.\n"
        "   - Only choose ',' if the comma specifically separates fractional cents at the very end of the number (e.g., '1.500,00').\n"
        "Then use the `execute_money_formatting` MCP tool to apply the formatting.\n"
        "You MUST pass all required parameters: file_path, col_name, is_mixed_currency, detected_currency, scale_decision, and decimal_separator."
    ),
    model="gpt-4o-2024-08-06",
)

# --- NAME AGENT ---
name_agent = Agent(
    name="Name Agent",
    instructions=(
        "You are a precise text standardization agent specializing in proper nouns.\n"
        "When given a file and a column name, first use `read_column_sample` to look at the data (request at least 10 rows).\n"
        "Your task:\n"
        "1. Determine if this column primarily contains 'Human Names' or 'Locations/Other' (like cities, states, companies).\n"
        "2. If it is 'Human Names', deduce the dominant structural format.\n"
        "   - Are they mostly 'First Last' (e.g., John Smith)?\n"
        "   - Are they mostly 'Last First' (e.g., Smith John)?\n"
        "   - NOTE: If you see ambiguous names (like 'Harper Taylor'), look at the other names in the sample to deduce the pattern.\n"
        "3. If it is 'Locations/Other', select 'N/A' for the format.\n"
        "Then use the `execute_name_formatting` MCP tool to apply the formatting.\n"
        "You MUST pass all required parameters: file_path, col_name, entity_type, and dominant_format."
    ),
    model="gpt-4o-2024-08-06",
)

# --- DESCRIPTION AGENT (NEW — matches Cell 1's dataset_description_agent) ---
description_agent = Agent(
    name="Description Agent",
    instructions=(
        "You are an expert data analyst and documentation agent.\n"
        "When given a file path, use `read_data_sample` to get a sample of the cleaned data.\n"
        "Then use `get_columns` to get all column names.\n\n"
        "Your task is to generate a comprehensive data dictionary:\n"
        "1. Write a 1-2 sentence `general_summary` of what this dataset represents.\n"
        "2. For every single column in the dataset, create an entry with:\n"
        "   - 'Feature Name': The exact column name.\n"
        "   - 'Conceptual Data Type': e.g., Categorical, Datetime, Continuous Numeric, Text.\n"
        "   - 'Description': A clear, concise description of what the data represents.\n\n"
        "Then use the `execute_dataset_description` MCP tool to save the description as a second sheet.\n"
        "You MUST pass:\n"
        "   - file_path: the path to the file\n"
        "   - general_summary: your 1-2 sentence summary\n"
        "   - features_json: a JSON string representing a list of objects with keys 'Feature Name', 'Conceptual Data Type', 'Description'\n\n"
        "Example features_json: '[{\"Feature Name\": \"Age\", \"Conceptual Data Type\": \"Continuous Numeric\", \"Description\": \"The age of the person in years.\"}]'"
    ),
    model="gpt-4o-2024-08-06",
)

# 4. Define the Orchestrator
async def run_agentic_pipeline(file_path: str):
    server_path = str(Path("data_formatting_server.py").resolve())
    python_executable = sys.executable
    
    async with MCPServerStdio(
        name="Data Formatting Tools",
        params={
            "command": python_executable,
            "args": [server_path],
        },
    ) as server:
        
        orchestrator = Agent(
            name="Data Pipeline Orchestrator",
            instructions=(
                "You are the orchestrator of a data cleaning pipeline. Follow these steps EXACTLY in order:\n\n"
                "STEP 1 — SCOUT: Use the `header_agent` to detect and apply the correct header row and crop empty columns.\n"
                "   Pass a message like: 'Detect the header and crop the file \"<file_path>\"'.\n\n"
                "STEP 2 — SWEEP: Use the `na_agent` to scan for and clean custom NA placeholders, empty rows, and empty columns.\n"
                "   Pass a message like: 'Clean missing data in the file \"<file_path>\"'.\n\n"
                "STEP 3 — READ: Use the `reader_agent` to classify ALL columns in the file. Pass the file_path to it.\n"
                "   The reader_agent will return a JSON mapping of column names to types.\n\n"
                "STEP 4 — ROUTE: Process EACH column one by one in order, based on its classified type:\n"
                "   - 'time': Delegate to `time_agent`. Pass a message like: 'Format the time column \"<col_name>\" in file \"<file_path>\"'.\n"
                "   - 'money': Delegate to `money_agent`. Pass a message like: 'Format the money column \"<col_name>\" in file \"<file_path>\"'.\n"
                "   - 'name': Delegate to `name_agent`. Pass a message like: 'Format the name column \"<col_name>\" in file \"<file_path>\"'.\n"
                "   - 'int': Directly use the `execute_int_formatting` MCP tool with file_path and col_name. Do NOT use an agent.\n"
                "   - 'float': Directly use the `execute_float_formatting` MCP tool with file_path and col_name. Do NOT use an agent.\n"
                "   - 'string' or 'unknown': Bypass — do nothing, these require no formatting.\n\n"
                "STEP 5 — DESCRIBE: Use the `description_agent` to generate a data dictionary and save it as a second sheet.\n"
                "   Pass a message like: 'Generate a dataset description for the file \"<file_path>\"'.\n\n"
                "CRITICAL RULES:\n"
                "   - You MUST execute ALL 5 steps in the exact order above.\n"
                "   - For 'time', 'money', and 'name', you MUST delegate to the respective agents and NOT call the MCP formatting tools directly.\n"
                "   - For 'int' and 'float', you MUST call the MCP tools directly and NOT delegate to agents.\n"
                "   - Always pass BOTH file_path AND col_name when delegating or calling tools.\n"
                "   - Process columns in order from left to right.\n\n"
                "STEP 6: After all steps are complete, summarize the actions taken."
            ),
            tools=[
                get_columns,
                read_data_sample,
                header_agent.as_tool(
                    tool_name="header_agent",
                    tool_description="Detect the true header row and starting column, then re-read and crop the file accordingly. Pass the file_path."
                ),
                na_agent.as_tool(
                    tool_name="na_agent",
                    tool_description="Scan for custom NA placeholder strings and clean empty rows/columns. Pass the file_path."
                ),
                reader_agent.as_tool(
                    tool_name="reader_agent", 
                    tool_description="Classify the data types of ALL columns in the file. Pass the file_path. Returns a JSON mapping of column_name -> type."
                ),
                time_agent.as_tool(
                    tool_name="time_agent", 
                    tool_description="Format a time/date column. You MUST include both the file_path and col_name in your message to this agent."
                ),
                money_agent.as_tool(
                    tool_name="money_agent", 
                    tool_description="Format a money/financial column. You MUST include both the file_path and col_name in your message to this agent."
                ),
                name_agent.as_tool(
                    tool_name="name_agent", 
                    tool_description="Format a name/proper noun column. You MUST include both the file_path and col_name in your message to this agent."
                ),
                description_agent.as_tool(
                    tool_name="description_agent",
                    tool_description="Generate a data dictionary for the cleaned dataset and save it as a second sheet. Pass the file_path."
                ),
            ],
            mcp_servers=[server],
            model="gpt-4o-2024-08-06",
        )
        
        # Give sub-agents access to MCP server and local tools
        header_agent.mcp_servers = [server]
        header_agent.tools = []
        
        na_agent.mcp_servers = [server]
        na_agent.tools = []
        
        time_agent.mcp_servers = [server]
        time_agent.tools = [read_column_sample]
        
        money_agent.mcp_servers = [server]
        money_agent.tools = [read_column_sample]
        
        name_agent.mcp_servers = [server]
        name_agent.tools = [read_column_sample]
        
        description_agent.mcp_servers = [server]
        description_agent.tools = [read_data_sample, get_columns]

        print(f"--- STARTING AGENTIC PIPELINE (MCP + SDK) for {file_path} ---")
        result = await Runner.run(
            orchestrator,
            f"Please analyze and format the data in '{file_path}'. Process every column."
        )
        print("\n[Orchestrator Summary]:")
        print(result.final_output)

# 5. Run the pipeline
test_file_mcp = "cleaned_test_pipeline_mcp.xlsx"
pd.DataFrame({
    "Event Date": ["first of january 2016", "january second 2016", "yesterday"],
    "Revenue": ["100 million dollars", "200000000", "300 mil eur"],
    "entities": [5, 10, 15.0],
    "Customer": ["Alice", "Bob", "Charlie SMITH"]
}).to_excel(test_file_mcp, index=False)

# Run the async function
await run_agentic_pipeline(test_file_mcp)

--- STARTING AGENTIC PIPELINE (MCP + SDK) for cleaned_test_pipeline_mcp.xlsx ---

[Orchestrator Summary]:
Here's a summary of the actions taken for each column in 'cleaned_test_pipeline_mcp.xlsx':

1. **Event Date**: Identified as a time column. Contains non-standardized entries like "first of january 2016" and "yesterday," which require conversion to a standard date format `%d/%m/%Y`. 

2. **Revenue**: Identified as a money column. Successfully formatted with mixed currencies using a scale of millions.

3. **entities**: Identified as an int column. Successfully formatted to integers.

4. **Customer**: Identified as a name column. Successfully formatted with the dominant "First Last" name structure.

For the Event Date, ambiguous entries like "yesterday" can be converted to explicit dates based on today's date. Would you like to proceed with that conversion?

[Orchestrator Summary]:
Here's a summary of the actions taken for each column in 'cleaned_test_pipeline_mcp.xlsx':

1. **Event D